# Build a historical data set for testing

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from ez_bsm.utils import SingleExpirationChain

In [2]:
raw_data = pd.read_csv(r'C:\Users\aspei\Dropbox\Options\historical_data_archive\allspx\SPX_2015.csv',
                       parse_dates=['expiration','quotedate'],
                      )
raw_data['is_call'] = (raw_data.type=='call')
raw_data['dte'] = (raw_data.expiration - raw_data.quotedate).dt.days

In [8]:
close_dates = pd.DatetimeIndex(raw_data.quotedate.unique())

In [14]:
all_chains = []
for close_date in close_dates[:60]:
    print(close_date)
    chains = raw_data[raw_data.quotedate==close_date].copy()
    chains = chains[(chains.dte>30) & (chains.dte<220)]
    chains = chains[chains.bid>0]
    chains = chains[chains.strike % 25 == 0]
    for expiration, chain in chains.groupby('expiration'):
        assert all(chain.groupby(['type','strike']).count().max(axis=1)==1)
        sec = SingleExpirationChain(chain.is_call, chain.strike,
                                    mark=0.5*(chain.bid+chain.ask),
                                    expiration=expiration, 
                                    is_am=True, close_date=close_date)
        sec.compute_all()
        all_chains.append(sec.as_dataframe())

2015-01-02 00:00:00
2015-01-05 00:00:00
2015-01-06 00:00:00
2015-01-07 00:00:00
2015-01-08 00:00:00
2015-01-09 00:00:00
2015-01-12 00:00:00
2015-01-13 00:00:00
2015-01-14 00:00:00
2015-01-15 00:00:00
2015-01-16 00:00:00
2015-01-20 00:00:00
2015-01-21 00:00:00
2015-01-22 00:00:00
2015-01-23 00:00:00
2015-01-26 00:00:00
2015-01-27 00:00:00
2015-01-28 00:00:00
2015-01-29 00:00:00
2015-01-30 00:00:00
2015-02-02 00:00:00
2015-02-03 00:00:00
2015-02-04 00:00:00
2015-02-05 00:00:00
2015-02-06 00:00:00
2015-02-09 00:00:00
2015-02-10 00:00:00
2015-02-11 00:00:00
2015-02-12 00:00:00
2015-02-13 00:00:00
2015-02-17 00:00:00
2015-02-18 00:00:00
2015-02-19 00:00:00
2015-02-20 00:00:00
2015-02-23 00:00:00
2015-02-24 00:00:00
2015-02-25 00:00:00
2015-02-26 00:00:00
2015-02-27 00:00:00
2015-03-02 00:00:00
2015-03-03 00:00:00
2015-03-04 00:00:00
2015-03-05 00:00:00
2015-03-06 00:00:00
2015-03-09 00:00:00
2015-03-10 00:00:00
2015-03-11 00:00:00
2015-03-12 00:00:00
2015-03-13 00:00:00
2015-03-16 00:00:00


In [15]:
result = pd.concat(all_chains, axis=0, ignore_index=True)

In [23]:
result.to_csv(r'tests/spx_sample_q1_2015.csv', index=None)